In [8]:
import pandas as pd
import datetime as dt 
import requests
from bs4 import BeautifulSoup
import sys
import os
from datetime import datetime
import argparse
import json
import re
import time
from random import randint
from selenium import webdriver
import numpy as np
import ast
import random
import itertools
from selenium.webdriver.support.ui import WebDriverWait

ModuleNotFoundError: No module named 'requests'

In [1]:
# Added 2033
from bs4 import BeautifulSoup
import requests

In [2]:
base_url = 'https://www.muscleandstrength.com/workouts/{}'
request_agent = 'Mozilla/5.0 Chrome/47.0.2526.106 Safari/537.36'

In [3]:
def getSoup(url):
    return BeautifulSoup(requests.get(url, headers={'User-Agent':request_agent}).text,'lxml')

In [5]:
soup = getSoup("https://bodyspace.bodybuilding.com/workouts/viewworkoutlog/drmom4u/5b612bcb10ed7c6d82eb1859")

In [8]:
def scrapeWorkout(url):
    soup = getSoup(url)
    workout = dict()

    muslcesWorked = soup.find("div",{"class":"musclesWorked"}).find("span",{"class","value"})
    muscles = muslcesWorked.text.split(", ")
    workout["Muscles"] = muscles

    time = soup.find("span",{"wicketpath":"logResultsPanel_workoutSummary_totalWorkoutTime"}).text
    tot = int(time.split(":")[0])*60+int(time.split(":")[1])
    workout["Time"] = tot

    descriptions = soup.findAll("div",{"class":"exercise-overview"})
    sets = soup.findAll("div",{"class":"exercise-details"})


    exerc = []
    for i in range(len(descriptions)):
        if len(descriptions[i].findAll('div',{"class":"superset-title-remove"})) == 1:
            curr = descriptions[i]
            superset = []
            for ex in curr.findAll("div",{"class","superSetExercise"}):
                temp = dict()
                temp["Name"] = (ex.find("h3").text)
                superset.append(temp)
                
            count = len(superset)
            
            test = []
            for j in range(count):
                test.append([])
            
            for st in sets[i].findAll("div",{"class":"set"}):
                s = st.findAll("label",{"class":"left-label"})
                inpwrap = st.findAll("div",{"class":"inputWrapper"})
            
                if len(inpwrap) > len(superset):
                    test = scrapeWorkout2(sets[i],test)
                    break
                else: 
                    for r in range(count):
                        temp = dict()
                        set_type = s[r].text.strip().split("\n")[0].strip()
                        if set_type == 'WEIGHT/REPS':
                            try:
                                weight = (inpwrap[r].findAll("div",{"class":"logLabelContainer"})[0].find("span",{"class":"value"}).text)
                                reps = inpwrap[r].findAll("div",{"class":"logLabelContainer"})[1].find("span",{"class":"value"}).text
                                temp["Weight"] = weight
                                temp["Reps"] = reps
                            except:
                                reps = inpwrap[r].findAll("div",{"class":"logLabelContainer"})[0].find("span",{"class":"value"}).text
                                temp["Reps"] = reps
                            test[r].append(temp)
                        elif set_type == 'REPS':
                            reps = inpwrap[r].findAll("div",{"class":"logLabelContainer"})[0].find("span",{"class":"value"}).text
                            temp["Reps"] = reps
                            test[r].append(temp)
                        elif set_type == 'TIME':
                            totSeconds = 0
                            for wx in inpwrap[r].findAll("div",{"class":"logLabelContainer"}):
                                if wx.text.split("\n")[1] == 'hr':
                                    totSeconds += int(wx.text.split("\n")[0])*3600
                                elif wx.text.split("\n")[1] == 'min':
                                    totSeconds += int(wx.text.split("\n")[0])*60
                                elif wx.text.split("\n")[1] == 'sec':
                                    totSeconds += int(wx.text.split("\n")[0])
                            temp['Time'] = totSeconds
            
            for w in range(count):
                superset[w]['Sets'] = test[w]
                superset[w]['No-Sets'] = len(test[w])
            exerc.append(superset)
            
            
        else:  
            ex = dict()
            name = (descriptions[i].find("div",{"class":"exercise-info"}).find("h3").text)
            ex["Name"] = name
            setEx = []
            for st in sets[i].findAll("div",{"class":"set"}):
                temp = dict()
                set_type = st.find("label",{"class":"left-label"}).text.strip().split("\n")[0]
                wxr = st.find("div",{"class":"inputWrapper"}).findAll("div",{"class":"logLabelContainer"})

                if set_type == 'TIME':
                    totSeconds = 0
                    for wx in wxr:
                        if wx.text.split("\n")[1] == 'hr':
                            totSeconds += int(wx.text.split("\n")[0])*3600
                        elif wx.text.split("\n")[1] == 'min':
                            totSeconds += int(wx.text.split("\n")[0])*60
                        elif wx.text.split("\n")[1] == 'sec':
                            totSeconds += int(wx.text.split("\n")[0])
                    temp['Time'] = totSeconds
                elif set_type == 'WEIGHT/REPS':
                    try:
                        weight = wxr[0].find("span",{"class":"value"}).text
                        reps = wxr[1].find("span",{"class":"value"}).text
                        temp["Weight"] = weight
                        temp["Reps"] = reps
                    except:
                        reps = wxr[0].find("span",{"class":"value"}).text
                        temp["Reps"] = reps
                elif set_type == 'REPS':
                    reps = wxr[0].find("span",{"class":"value"}).text
                    temp["Reps"] = reps
                    
                setEx.append(temp)
            ex["Sets"] = setEx
            ex["No-Sets"] = len(setEx)
            exerc.append(ex)
    workout["Exercises"] = exerc
    workout["URL"] = url
    workout["Rating"] = int(soup.find("span",{"class","bigRating"}).text.strip())/10
    return workout

In [9]:
url = "https://bodyspace.bodybuilding.com/workouts/viewworkoutlog/drmom4u/5b612bcb10ed7c6d82eb1859"
scrapeWorkout(url)


{'Muscles': ['Abdominals', 'Middle Back', 'Quadriceps', 'Lats'],
 'Time': 62,
 'Exercises': [{'Name': 'Wide-Grip Lat Pulldown',
   'Sets': [{'Weight': '50', 'Reps': '25'},
    {'Weight': '60', 'Reps': '25'},
    {'Weight': '70', 'Reps': '20'},
    {'Weight': '50', 'Reps': '25'}],
   'No-Sets': 4},
  {'Name': 'Seated Cable Rows',
   'Sets': [{'Weight': '60', 'Reps': '25'},
    {'Weight': '65', 'Reps': '25'},
    {'Weight': '70', 'Reps': '20'},
    {'Weight': '55', 'Reps': '25'}],
   'No-Sets': 4},
  {'Name': 'Goblet Squat',
   'Sets': [{'Weight': '20', 'Reps': '10'},
    {'Weight': '20', 'Reps': '10'},
    {'Weight': '20', 'Reps': '10'}],
   'No-Sets': 3},
  {'Name': 'Cable Crunch',
   'Sets': [{'Weight': '95', 'Reps': '25'},
    {'Weight': '105', 'Reps': '20'},
    {'Weight': '105', 'Reps': '20'}],
   'No-Sets': 3}],
 'URL': 'https://bodyspace.bodybuilding.com/workouts/viewworkoutlog/drmom4u/5b612bcb10ed7c6d82eb1859',
 'Rating': 0.7}

In [7]:
def scrapeWorkout2(sets,test):
    set_count = 0
    overall = []
    for st in sets.findAll('div',{"class","set"}):
        set_l = []
        for sbody in st.findAll("div",{"class","set-body"}):
            ex = dict()
            for row in sbody.findAll("div",{"class","set-row"}):
                temp = parse_row(row)
                if temp[0] == 'T':
                    ex["Time"] = temp[1]
                elif temp[0] == 'R':
                    ex["Reps"] = temp[1]
                elif temp[1] == 'WR':
                    ex["Weight"] = temp[1]
                    ex["Reps"] = temp[2]
            set_l.append(ex)
        overall.append(set_l)
        set_count+=1
    
    for i in range(len(test)):
        for j in range(len(overall)):
            test[i].append(overall[j][i])
            
    return test

In [8]:
def parse_row(row):
    label = row.find("label",{"class","left-label"}).text.strip().replace(":","")
    if "\n" in label:
        label = label.split("\n")[0].strip()   
    if label == 'TIME':
        vals = row.findAll("span",{"class","value"})
        total = int(vals[0].text.strip())*3600 + int(vals[1].text.strip())*60 + int(vals[2].text.strip())
        return ("T", total)
    if label == 'REPS':
        value = int(row.find("span",{"class","value"}).text.strip())
        return ("R", value)
    if label == "WEIGHT/REPS":
        try:
            vals = row.findAll("span",{"class","value"})
            return ("WR",float(vals[0].text.strip()), int(vals[1].text.strip()))
        except:
            value = int(row.find("span",{"class","value"}).text.strip())
            return ("R", value)
    else:
        print(row)

In [9]:
def scrape_workout_row(row):
    df = pd.DataFrame(columns=['Height', 'Weight', 'Body_Fat', 'Age', 'Gender', 'Member_Since', 'Goal',
       'Link','Rating', 'Muscles', 'Time','Exercises'])
    h = row.Height
    w = row.Weight
    bf = row.Body_Fat
    a = row.Age
    g = row.Gender
    ms = row.Member_Since
    goal = row.Goal
    for l in row.Links:
        try:
            test = scrapeWorkout(l)
            time = test['Time']
            muscles = test['Muscles']
            rat = test['Rating']
            exercises = test['Exercises']
            new = {'Height':h, 'Weight':w, 'Body_Fat':bf, 'Age':a, 'Gender':g, 'Member_Since':ms, 'Goal':goal,
           'Link':l,'Rating':rat, 'Muscles':muscles, 'Time':time,'Exercises':exercises}
            df = df.append(new,ignore_index = True)
        except:
            continue
    return df


In [10]:
def scrape_file(path,fname):
    data = pd.read_csv('../../Data/Bodyspace/WorkoutLinks_18_Members_Full.csv',encoding = 'utf-8',engine='python')
    data.Links = data.Links.apply(ast.literal_eval)
    
    finished = []
    dirName = path+fname[:-4]
    if not os.path.exists(dirName):
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ")
    else:    
        print("Directory " , dirName ,  " already exists")
        for r, d, f in os.walk(dirName+"/"):
            for file in f:
                if '.csv' in file:
                    finished.append(file[:-4])
    
    failed =[]
    for idx,row in data.iterrows():
        if str(idx) in finished:
            print("User - ",idx, "Complete")
        else:
            print("User - ",idx, "Scraping")
            df = pd.DataFrame(columns=['Height', 'Weight', 'Body_Fat', 'Age', 'Gender', 'Member_Since', 'Goal',
       'Link','Rating', 'Muscles', 'Time','Exercises'])
            
            try:
                temp = scrape_workout_row(row)
                df = df.append(temp, ignore_index=True)
                df.to_csv(dirName+"/{}.csv".format(idx),sep="¬",quotechar='"',encoding = 'utf-8')
            except:
                failed.append(idx)
                print("\t-Failed")
           
            
    return data

In [11]:
# add percent in brackets after
# add total workouts scarped counter

In [ ]:
path = 'Data/Bodyspace/'
fname = 'WorkoutLinks_18_Members_Full.csv'
scrape_file(path,fname).head()

Directory  ../../Data/Bodyspace/WorkoutLinks_18_Members_Full  already exists
User -  0 Complete
User -  1 Complete
User -  2 Complete
User -  3 Complete
User -  4 Complete
User -  5 Complete
User -  6 Complete
User -  7 Complete
User -  8 Complete
User -  9 Complete
User -  10 Complete
User -  11 Complete
User -  12 Complete
User -  13 Complete
User -  14 Complete
User -  15 Complete
User -  16 Complete
User -  17 Complete
User -  18 Complete
User -  19 Complete
User -  20 Complete
User -  21 Complete
User -  22 Complete
User -  23 Complete
User -  24 Complete
User -  25 Complete
User -  26 Complete
User -  27 Complete
User -  28 Complete
User -  29 Complete
User -  30 Complete
User -  31 Complete
User -  32 Complete
User -  33 Complete
User -  34 Complete
User -  35 Complete
User -  36 Complete
User -  37 Complete
User -  38 Complete
User -  39 Complete
User -  40 Complete
User -  41 Complete
User -  42 Complete
User -  43 Complete
User -  44 Complete
User -  45 Complete
User -  46 Co

User -  472 Complete
User -  473 Complete
User -  474 Complete
User -  475 Complete
User -  476 Complete
User -  477 Complete
User -  478 Complete
User -  479 Complete
User -  480 Complete
User -  481 Complete
User -  482 Complete
User -  483 Complete
User -  484 Complete
User -  485 Complete
User -  486 Complete
User -  487 Complete
User -  488 Complete
User -  489 Complete
User -  490 Complete
User -  491 Complete
User -  492 Complete
User -  493 Complete
User -  494 Complete
User -  495 Complete
User -  496 Complete
User -  497 Complete
User -  498 Complete
User -  499 Complete
User -  500 Complete
User -  501 Complete
User -  502 Complete
User -  503 Complete
User -  504 Complete
User -  505 Complete
User -  506 Complete
User -  507 Complete
User -  508 Complete
User -  509 Complete
User -  510 Complete
User -  511 Complete
User -  512 Complete
User -  513 Complete
User -  514 Complete
User -  515 Complete
User -  516 Complete
User -  517 Complete
User -  518 Complete
User -  519 C

User -  972 Complete
User -  973 Complete
User -  974 Complete
User -  975 Complete
User -  976 Complete
User -  977 Complete
User -  978 Complete
User -  979 Complete
User -  980 Complete
User -  981 Complete
User -  982 Complete
User -  983 Complete
User -  984 Complete
User -  985 Complete
User -  986 Complete
User -  987 Complete
User -  988 Complete
User -  989 Complete
User -  990 Complete
User -  991 Complete
User -  992 Complete
User -  993 Complete
User -  994 Complete
User -  995 Complete
User -  996 Complete
User -  997 Complete
User -  998 Complete
User -  999 Complete
User -  1000 Complete
User -  1001 Complete
User -  1002 Complete
User -  1003 Complete
User -  1004 Complete
User -  1005 Complete
User -  1006 Complete
User -  1007 Complete
User -  1008 Complete
User -  1009 Complete
User -  1010 Complete
User -  1011 Complete
User -  1012 Complete
User -  1013 Complete
User -  1014 Complete
User -  1015 Complete
User -  1016 Complete
User -  1017 Complete
User -  1018 Com

User -  1531 Complete
User -  1532 Complete
User -  1533 Complete
User -  1534 Complete
User -  1535 Complete
User -  1536 Complete
User -  1537 Complete
User -  1538 Complete
User -  1539 Complete
User -  1540 Complete
User -  1541 Complete
User -  1542 Complete
User -  1543 Complete
User -  1544 Complete
User -  1545 Complete
User -  1546 Complete
User -  1547 Complete
User -  1548 Complete
User -  1549 Complete
User -  1550 Complete
User -  1551 Complete
User -  1552 Complete
User -  1553 Complete
User -  1554 Complete
User -  1555 Complete
User -  1556 Complete
User -  1557 Complete
User -  1558 Complete
User -  1559 Complete
User -  1560 Complete
User -  1561 Complete
User -  1562 Complete
User -  1563 Complete
User -  1564 Complete
User -  1565 Complete
User -  1566 Complete
User -  1567 Complete
User -  1568 Complete
User -  1569 Complete
User -  1570 Complete
User -  1571 Complete
User -  1572 Complete
User -  1573 Complete
User -  1574 Complete
User -  1575 Complete
User -  15

User -  2006 Complete
User -  2007 Complete
User -  2008 Complete
User -  2009 Complete
User -  2010 Complete
User -  2011 Complete
User -  2012 Complete
User -  2013 Complete
User -  2014 Complete
User -  2015 Complete
User -  2016 Complete
User -  2017 Complete
User -  2018 Complete
User -  2019 Complete
User -  2020 Complete
User -  2021 Complete
User -  2022 Complete
User -  2023 Complete
User -  2024 Complete
User -  2025 Complete
User -  2026 Complete
User -  2027 Complete
User -  2028 Complete
User -  2029 Complete
User -  2030 Complete
User -  2031 Complete
User -  2032 Complete
User -  2033 Complete
User -  2034 Complete
User -  2035 Complete
User -  2036 Complete
User -  2037 Complete
User -  2038 Complete
User -  2039 Complete
User -  2040 Complete
User -  2041 Complete
User -  2042 Complete
User -  2043 Complete
User -  2044 Complete
User -  2045 Complete
User -  2046 Complete
User -  2047 Complete
User -  2048 Complete
User -  2049 Complete
User -  2050 Complete
User -  20